In [1]:
pip install opencv-python numpy pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

# --- Load Object Detection Model ---
thres = 0.5
classnames = []
classfile = "coco.pbtxt"

with open(classfile, 'rt') as f:
    classnames = f.read().rstrip("\n").split("\n")

configpath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightspath = 'frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightspath, configpath)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

# --- Tkinter GUI ---
root = tk.Tk()
root.title("Object Detection GUI")
root.geometry("800x600")

try:
    bg_image = Image.open("back.jpg")
    bg_image = bg_image.resize((800, 600), Image.LANCZOS)
    bg_photo = ImageTk.PhotoImage(bg_image)

    bg_label = tk.Label(root, image=bg_photo)
    bg_label.image = bg_photo  # Keep reference to prevent garbage collection
    bg_label.place(relwidth=1, relheight=1)

except Exception as e:
    print(f"Error loading background image: {e}")

# --- Function: Live Object Detection ---
def live_detection():
    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)

    while True:
        success, img = cap.read()
        classIds, confs, bbox = net.detect(img, confThreshold=thres)

        if len(classIds) != 0:
            for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
                cv2.rectangle(img, box, color=(0, 255, 2), thickness=3)
                cv2.putText(img, classnames[classId - 1].upper(), (box[0] + 10, box[1] + 30),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(img, str(round(confidence * 100, 2)), (box[0] + 200, box[1] + 30),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 5), 2)

        cv2.imshow("Live Object Detection", img)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# --- Function: Static Image Object Detection ---
def detect_from_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.png;*.jpeg")])
    
    if not file_path:
        return
    
    img = cv2.imread(file_path)
    classIds, confs, bbox = net.detect(img, confThreshold=thres)

    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
            cv2.rectangle(img, box, color=(0, 255, 2), thickness=3)
            cv2.putText(img, classnames[classId - 1].upper(), (box[0] + 10, box[1] + 30),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(img, str(round(confidence * 100, 2)), (box[0] + 200, box[1] + 30),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 5), 2)

    cv2.imshow("Static Image Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# --- Buttons ---
upload_button = tk.Button(root, text="Upload Image for Detection", command=detect_from_image, font=("Arial", 14), bg="lightblue")
upload_button.pack(pady=20)

live_button = tk.Button(root, text="Start Live Object Detection", command=live_detection, font=("Arial", 14), bg="lightgreen")
live_button.pack(pady=20)

# Run Tkinter
root.mainloop()
